In [1]:
!nvidia-smi

Sat Mar  1 13:19:38 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.230.02             Driver Version: 535.230.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 80GB HBM3          On  | 00000000:1A:00.0 Off |                    0 |
| N/A   26C    P0              69W / 700W |      0MiB / 81559MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [8]:
!apt-get install zip unzip


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  unzip zip
0 upgraded, 2 newly installed, 0 to remove and 62 not upgraded.
Need to get 350 kB of archives.
After this operation, 930 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 unzip amd64 6.0-26ubuntu3.2 [175 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 zip amd64 3.0-12build2 [176 kB]
Fetched 350 kB in 0s (1235 kB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package unzip.
(Reading database ... 15512 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-26ubuntu3.2_amd64.deb ...
Unpacking unzip (6.0-26ubuntu3.2) ...
Selecting previously unselected package zip.
Preparing to unpack .../zip_3.0-12build2_amd64.deb ...
Unpacking zip (3.0-12build2) ...
Setting up unzip (6.0-26ubuntu3.2) 

In [6]:
#%cd zeynebnk
#!unzip LLaDA_eval.zip
%cd content/LLaDA/

/persistent/joeli/zeynebnk/content/LLaDA


In [7]:
dir = ""
num_gpus = 8
bench_data_path = "gsm8k.txt"
output_dir =  "../../outputs/"
remasking_method = "low_confidence"

import subprocess

def runsubprocess(cmd):
  subprocess.run(cmd, shell=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)


In [8]:

prl_cmd = f"python prep_parallel.py --data {dir + bench_data_path} --output_head {dir + 'chunk'} --num_gpu {num_gpus}"
runsubprocess(prl_cmd)


In [9]:
import os

In [ ]:
def run_on_gpu(script_path, gpu_id, args):
    """Run a Python script on a specific GPU with arguments."""
    env = os.environ.copy()
    env["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    
    cmd = ["python", script_path] + args
    print(f"Starting {script_path} with args {args} on GPU {gpu_id}")
    process = subprocess.Popen(cmd, env=env)
    return process

ps = []
for gpu_id in range(num_gpus):
  ## ADD CODE TO MAKE RUN IN PARALLEL
  # pred_cmd = f"python gen_batch_preds.py --filepath {dir + 'chunk' + "_" + str(i) + ".txt"} --output_path {output_dir + "_" + str(i) + "preds.txt"} --remasking_method {remasking_method}"
  # runsubprocess(pred_cmd)
    
    script_path = "gen_batch_preds.py"
    args = [
        "--filepath", f"{dir}chunk_{gpu_id}.txt", 
        "--output_path", f"{output_dir}_{gpu_id}preds.txt", 
        "--remasking_method", remasking_method,

        "--gen_len",'256',
        "--steps",'128',
        "--block_size",'8'
    ]
    process = run_on_gpu(script_path, gpu_id, args)
    ps.append(process)

for p in ps:
    p.wait()
    

Starting gen_batch_preds.py with args ['--filepath', 'chunk_0.txt', '--output_path', '../../outputs/_0preds.txt', '--remasking_method', 'low_confidence', '--gen_len', '256', '--steps', '128', '--block_size', '8'] on GPU 0
Starting gen_batch_preds.py with args ['--filepath', 'chunk_1.txt', '--output_path', '../../outputs/_1preds.txt', '--remasking_method', 'low_confidence', '--gen_len', '256', '--steps', '128', '--block_size', '8'] on GPU 1
Starting gen_batch_preds.py with args ['--filepath', 'chunk_2.txt', '--output_path', '../../outputs/_2preds.txt', '--remasking_method', 'low_confidence', '--gen_len', '256', '--steps', '128', '--block_size', '8'] on GPU 2
Starting gen_batch_preds.py with args ['--filepath', 'chunk_3.txt', '--output_path', '../../outputs/_3preds.txt', '--remasking_method', 'low_confidence', '--gen_len', '256', '--steps', '128', '--block_size', '8'] on GPU 3
Starting gen_batch_preds.py with args ['--filepath', 'chunk_4.txt', '--output_path', '../../outputs/_4preds.txt'

A new version of the following files was downloaded from https://huggingface.co/GSAI-ML/LLaDA-8B-Instruct:
- configuration_llada.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/GSAI-ML/LLaDA-8B-Instruct:
- configuration_llada.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/GSAI-ML/LLaDA-8B-Instruct:
- configuration_llada.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/GSAI-ML/LLaDA-8B-Instruct:
- configuration_llada.py
. Make sure to double-

In [9]:
!ls "../../outputs/"

_0preds.txt  _2preds.txt  _4preds.txt  _6preds.txt
_1preds.txt  _3preds.txt  _5preds.txt  _7preds.txt


In [22]:

## AFTER ALL PREDS ARE GENERATED
all_pred_data = []
for i in range(num_gpus):
  with open(output_dir + "_" + str(i) + "preds.txt", "r") as f:
    all_pred_data += eval(f.read())

with open(output_dir + "_all_preds.txt", "w") as f:
    f.write(str(all_pred_data))

eval_cmd = f"python evaluate_gsm.py --pred_data {output_dir + '_all_preds.txt'} --output_dir {output_dir}"
runsubprocess(eval_cmd)